# Cvičení 07b - Hugging Face
Hugging Face je otevřená platforma a komunita, která se rychle stala centrem pro výzkum a nasazení moderních modelů hlubokého učení (Deep Learning), zejména v oblasti zpracování přirozeného jazyka (NLP), ale nyní i počítačového vidění.

Jádrem platformy je knihovna transformers, která poskytuje rozsáhlý katalog předtrénovaných modelů state-of-the-art (jako jsou BERT, GPT, ViT, Diffusion Models) a umožňuje jejich snadné stahování, úpravu a spouštění pomocí frameworků PyTorch a TensorFlow.

Hugging Face usnadňuje vědcům a inženýrům sdílení modelů, datasetů a demoverzí, čímž výrazně urychluje přechod od akademického výzkumu k praktickým aplikacím.

## Krok 1. Instalace a importy

In [ ]:
!pip install diffusers transformers accelerate torch Pillow
print("Knihovny nainstalovány.")

import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

### Proč se importuje co:

**import torch** - Nezbytný pro práci s tenzory, výpočty na GPU (CUDA) a pro manipulaci s modely, které jsou v PyTorch implementovány.

**from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler** - Modul difuzních modelů: Toto je specializovaná knihovna, která poskytuje:
- **StableDiffusionPipeline**: Hlavní třída, která automatizuje celý proces generování (načítání VAE, U-Net, Text Encoderu, a jejich koordinaci).
- **DPMSolverMultistepScheduler**: Algoritmus, který řídí *proces odšumování*.

**import matplotlib.pyplot as plt** - Vizualizace pro vykreslení mezikroků difuze a finálního obrázku přímo v notebooku.

**from PIL import Image** - Manipulace s obrázky: Používá se pro načítání a ukládání obrázku v paměti a pro zpracování obrazového výstupu z VAE dekodéru.

**import numpy as np** - Práce s poli.

## 2. Načtení modelu a scheduleru

In [ ]:
# Načtení modelu SD 1.5 z Hugging Face
MODEL_ID = "runwayml/stable-diffusion-v1-5" 

# Nastavení scheduleru pro rychlejší generování
scheduler = DPMSolverMultistepScheduler.from_pretrained(MODEL_ID, subfolder="scheduler")

# Vytvoření Pipeline a přesun na GPU
pipe = StableDiffusionPipeline.from_pretrained(MODEL_ID, scheduler=scheduler, torch_dtype=torch.float16)
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe.to(device)

print(f"Stable Diffusion 1.5 Pipeline načteno na: {device}")

**StableDiffusionPipeline** je hlavní třída, která automaticky načte všechny komponenty modelu (VAE, U-Net a Text Encoder).

Argument **torch_dtype=torch.float16** přepíná model do poloviční přesnosti (FP16), což výrazně šetří paměť GPU a zrychluje výpočty.

**DPMSolverMultistepScheduler** -- Tento scheduler používá pokročilé metody řešení diferenciálních rovnic, což mu umožňuje dosáhnout vysoké kvality výsledku za výrazně menšího počtu kroků (např. 20–50 kroků) ve srovnání se staršími schedulery (které vyžadovaly 1000+ kroků). Zajišťuje tedy rychlost a kvalitu.

## Krok 2. Funkce pro zobrazení mezivýsledků

Využíváme 'callback' pro zachycení latentních stavů během odšumování.

In [27]:
def visualize_diffusion_process(pipe, prompt, num_inference_steps=50, steps_to_capture=10):
    """Generuje obrázek a vizualizuje mezikroky difuze."""
    
    latent_steps = []
    
    # Callback funkce, která se spustí po každém kroku scheduleru
    def capture_latent_callback(pipe, step_index, timestep, callback_kwargs):
        # Ukládáme stav po každém 'steps_to_capture' kroku (např. každých 10 iterací)
        if step_index % steps_to_capture == 0:
            # Získáme aktuální latentní stav
            latents = callback_kwargs['latents']
            
            # Dekódujeme latentní stav na pixely pomocí VAE Dekodéru
            with torch.no_grad():
                # VAE dekodér převádí latenty na finální obrázek.
                image = pipe.vae.decode(latents / pipe.vae.config.scaling_factor, return_dict=False)[0]
                
            # Přesun na CPU a normalizace [0, 1]
            image = pipe.image_processor.postprocess(image, output_type="pil")
            
            latent_steps.append((step_index, image[0]))
            
        return callback_kwargs

    print(f"Spouštím generování pro {num_inference_steps} kroků, ukládám každých {steps_to_capture} kroků.")
    
    # Spuštění generování s custom callbackem
    final_result = pipe(
        prompt=prompt,
        num_inference_steps=num_inference_steps,
        callback=capture_latent_callback,
        callback_steps=1, # Callback se volá po každém kroku
        output_type="pil"
    ).images[0]
    
    return final_result, latent_steps

## Krok 3: Spuštění

In [ ]:
PROMPT = "Steampunk coffe machine"
FINAL_STEPS = 50 
CAPTURE_STEP = 10 # Ukládáme každý 10. krok

final_image, intermediate_images = visualize_diffusion_process(pipe, PROMPT, FINAL_STEPS, CAPTURE_STEP)

### Výsledná vizualizace
Zobrazení mezikroků a finálního výsledku.

In [ ]:
# Počet uložených mezikroků + finální obrázek
num_plots = len(intermediate_images) + 1 
fig, axs = plt.subplots(1, num_plots, figsize=(15, 3))
fig.suptitle(f"Proces Difuze pro '{PROMPT[:40]}...' ({FINAL_STEPS} kroků)", fontsize=10)

# 1. Zobrazení mezikroků
for i, (step, img) in enumerate(intermediate_images):
    axs[i].imshow(img)
    axs[i].set_title(f"Krok {step}", fontsize=8)
    axs[i].axis('off')

# 2. Zobrazení finálního výsledku
axs[-1].imshow(final_image)
axs[-1].set_title(f"Finále ({FINAL_STEPS})", fontsize=8)
axs[-1].axis('off')

plt.tight_layout()
plt.show()